In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers datasets sentencepiece sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-c

In [3]:
import pandas as pd
from datasets import Dataset

# Load your dataset (adjust separator if needed)
df = pd.read_csv("/kaggle/input/dataset-ugce/UGCE_en2indic_10k.csv")
# Drop rows with missing translations
df = df[['text', 'hi_text']].dropna()

# Rename columns for compatibility
df = df.rename(columns={'text': 'translation_en', 'hi_text': 'translation_hi'})

# Convert to HuggingFace dataset
dataset = Dataset.from_pandas(df)

# Format as translation pair
dataset = dataset.map(lambda x: {'translation': {'en': x['translation_en'], 'hi': x['translation_hi']}})

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['translation_en', 'translation_hi', 'translation'],
    num_rows: 10000
})

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set tokenizer for English → Hindi
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "hi_IN"

2025-04-14 18:59:28.101268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744657168.333857      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744657168.402404      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    # Iterate over the list of dictionaries for the "translation" column
    inputs = [trans["en"] for trans in examples["translation"]]
    targets = [trans["hi"] for trans in examples["translation"]]
    
    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    
    # Tokenize the targets in the target context
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
!pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 97.9 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [8]:
import wandb
wandb.login(key="5c556254e04a65a22bedb25b2faac3fc1b5b5272")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saketvempaty to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
!pip install evaluate
import evaluate
from transformers import Seq2SeqTrainer
import numpy as np

# Load the BLEU metric
bleu_metric = evaluate.load("bleu")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Prepare references as a list of lists
    references = [[label] for label in decoded_labels]

    # Compute BLEU score
    result = bleu_metric.compute(predictions=decoded_preds, references=references)

    # Return the BLEU score
    return {"bleu": result["bleu"]}


split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test (Eval) dataset size: {len(eval_dataset)}")
training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart50-hi",
    eval_strategy="epoch",         # Evaluate at the end of each epoch
    save_strategy="epoch",         # Save a checkpoint at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=1,            # Keep only the most recent checkpoint
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

from transformers import TrainerCallback
import random

import csv
from datetime import datetime

import numpy as np

class InferenceCallback(TrainerCallback):
    def __init__(self, tokenizer, model, eval_dataset, num_samples=10, output_file="inference_samples.csv"):
        self.tokenizer = tokenizer
        self.model = model
        self.eval_dataset = eval_dataset
        self.num_samples = num_samples
        self.output_file = output_file

        # Set seed here so it always selects the same samples
        #random.seed(42)
        #self.sample_indices = random.sample(range(len(self.eval_dataset)), self.num_samples)
        rng = np.random.default_rng(seed=42)
        self.sample_indices = rng.choice(len(self.eval_dataset), size=self.num_samples, replace=False).tolist()
        # Prepare consistent text samples
        self.sample_texts = [self.eval_dataset[i]['translation']['en'] for i in self.sample_indices]
        self.references = [self.eval_dataset[i]['translation']['hi'] for i in self.sample_indices]
        self.outputs = []

    def on_epoch_end(self, args, state, control, **kwargs):
        inputs = self.tokenizer(self.sample_texts, padding=True, truncation=True, return_tensors="pt").to(self.model.device)
        generated_ids = self.model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        predictions = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        for src, ref, pred in zip(self.sample_texts, self.references, predictions):
            self.outputs.append({
                "epoch": state.epoch,
                "source": src,
                "reference": ref,
                "prediction": pred
            })

        print(f"\n--- Inference After Epoch {state.epoch:.1f} ---")
        for src, pred, ref in zip(self.sample_texts, predictions, self.references):
            print(f"Source: {src}")
            print(f"Predicted: {pred}")
            print(f"Reference: {ref}")
            print("------")
        print("\n")

        return control

    def on_train_end(self, args, state, control, **kwargs):
        import pandas as pd
        df = pd.DataFrame(self.outputs)
        df.to_csv(self.output_file, index=False)
        print(f"\n✅ Inference samples saved to {self.output_file}\n")



# Instantiate the custom inference callback
inference_callback = InferenceCallback(
    tokenizer=tokenizer,
    model=model,
    eval_dataset=eval_dataset,
    num_samples=10  # You can adjust this to 3 or 4 samples
)

# Create the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Using the separate test set (1% of original data)
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[inference_callback]  # Add the inference callback here
)

# Train the model
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


Train dataset size: 9000
Test (Eval) dataset size: 1000


/tmp/ipykernel_31/3152583719.py:115: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Bleu
1,0.271600,0.272295,0.397197
2,0.187200,0.266218,0.414428
3,0.122000,0.271792,0.415650
4,0.079500,0.291028,0.409004
5,0.059700,0.299615,0.410370


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



--- Inference After Epoch 1.0 ---
Source: Suppose there are two partners X and Y, out of 70 say it is 1:1
Predicted: मान लें कि दो भागीदार X और Y हैं, 70 में से यह 1:1 है
Reference: मान लीजिए कि दो साझेदार X और Y हैं, 70 में से यह 1: 1 है
------
Source: This stage uses information from the crime scene to determine both behavioral and personal characteristics and build an outline for the criminal profile
Predicted: यह चरण अपराध स्थल से प्राप्त जानकारी का उपयोग व्यवहार संबंधी और व्यक्तिगत विशेषताओं को निर्धारित करने के लिए करता है और आपराधिक प्रोफाइल के लिए एक रूपरेखा बनाता है
Reference: यह चरण व्यवहार और व्यक्तिगत अभिलक्षणों दोनों को निर्धारित करने और आपराधिक प्रोफ़ाइल के लिए एक रूपरेखा बनाने के लिए अपराध स्थल से मिली जानकारी का उपयोग करता है
------
Source: They can assign the recovery material if necessary, define different paths for different learning goals
Predicted: वे आवश्यक होने पर पुन:प्राप्ति सामग्री को आवंटित कर सकते हैं, विभिन्न शिक्षण लक्ष्यों के लिए विभिन्न मार्गों को परिभा

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



--- Inference After Epoch 2.0 ---
Source: Suppose there are two partners X and Y, out of 70 say it is 1:1
Predicted: मान लीजिए दो साझेदार X और Y हैं, 70 में से यह 1:1 है
Reference: मान लीजिए कि दो साझेदार X और Y हैं, 70 में से यह 1: 1 है
------
Source: This stage uses information from the crime scene to determine both behavioral and personal characteristics and build an outline for the criminal profile
Predicted: यह मंच अपराध स्थल से प्राप्त जानकारी का उपयोग व्यवहार संबंधी और व्यक्तिगत विशेषताओं को निर्धारित करने और आपराधिक रूपरेखा के लिए एक रूपरेखा बनाने के लिए करता है
Reference: यह चरण व्यवहार और व्यक्तिगत अभिलक्षणों दोनों को निर्धारित करने और आपराधिक प्रोफ़ाइल के लिए एक रूपरेखा बनाने के लिए अपराध स्थल से मिली जानकारी का उपयोग करता है
------
Source: They can assign the recovery material if necessary, define different paths for different learning goals
Predicted: वे आवश्यक स्थिति में पुनर्प्राप्ति सामग्री निर्धारित कर सकते हैं, विभिन्न सीखने के लक्ष्यों के लिए विभिन्न मार्गों को परिभ

TrainOutput(global_step=11250, training_loss=0.18899773195054795, metrics={'train_runtime': 7768.6789, 'train_samples_per_second': 5.792, 'train_steps_per_second': 1.448, 'total_flos': 1.219010494464e+16, 'train_loss': 0.18899773195054795, 'epoch': 5.0})